# DATA

In [32]:
sem13_xml = "WSD_Unified_Evaluation_Datasets/semeval-2012-task-13-trial-data/data/semeval-2013-task-10-trial-data.xml"

In [33]:
import xml.etree.ElementTree as et 

xtree = et.parse(sem13_xml)
xroot = xtree.getroot()
d_tree = {}
c= 0

for child in xroot:
    tree = {}
    #print(child.tag, child.attrib, child.text)
    tree["id"] = child.get('id')
    tree['lemma'] = child.get('lemma')
    tree['partOfSpeech'] = child.get('partOfSpeech')
    tree['token'] = child.get('token')
    tree['tokenEnd'] = child.get('tokenEnd')
    tree['tokenStart'] = child.get('tokenStart')
    tree['text'] = child.text
    c+=1
    d_tree[c]=tree
    #print('\n')
#print(c,len(d_tree))       

# DATA Processing

In [34]:
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [35]:
def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    stopwords_alter = list(stopwords.words('english'))
    re = ['won',"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your",
          "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it",
          "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom","own"]
    for w in re:
        stopwords_alter.remove(w)
    for word in words:
        if word not in stopwords_alter:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(sample):
    sample = replace_contractions(sample)
    words = nltk.word_tokenize(sample)
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

#test the cleaning 
"""sample = d_tree[1]['text']
print(sample)
words = nltk.word_tokenize(sample)
print(words)
words = normalize(words)
print(words)"""

"sample = d_tree[1]['text']\nprint(sample)\nwords = nltk.word_tokenize(sample)\nprint(words)\nwords = normalize(words)\nprint(words)"

# getting synets from wordnet

In [36]:
import spacy 
from nltk.corpus import wordnet as wn
from collections import Counter 

In [37]:
def wordnet(word,tag):
    syns = wn.synsets(word, pos=tag)
    neigbhour = {}
    for s in syns:
        temp = []
        #print(s.name())
        #print(s.definition())
        definition = normalize(s.definition())
        temp.extend(definition)
        #print(definition)
        #print(s.examples())
        ex = [normalize(e) for e in s.examples()]
        for e in ex:
            temp.extend(e)
        #print(ex)
        #print(temp)
        synonyms = []
        antonyms = [] 
        for l in s.lemmas():
            #print(l.name(),l.key())
            if l.name() == word:
                name = l.key()
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())

        #print(set(synonyms))
        #print(set(antonyms))
        #print(Counter(temp))
        #print(list(set(temp)))
        neigbhour[name] = [list(set(temp)), s.definition()]
        #print('----------')
    return(neigbhour)

In [38]:
neighbour = wordnet('bank','n')
for key,value in neighbour.items():
    print(key,": ",value)
    print("---")

bank%1:17:01:: :  [['land', 'river', 'sloping', 'currents', 'slope', 'bank', 'watched', 'they', 'canoe', 'water', 'sat', 'pulled', 'beside', 'especially', 'body', 'he'], 'sloping land (especially the slope beside a body of water)']
---
bank%1:14:00:: :  [['accepts', 'cashed', 'activities', 'lending', 'holds', 'bank', 'deposits', 'channels', 'money', 'mortgage', 'home', 'check', 'financial', 'institution', 'my', 'he'], 'a financial institution that accepts deposits and channels the money into lending activities']
---
bank%1:17:00:: :  [['pile', 'earth', 'long', 'bank', 'ridge', 'huge'], 'a long ridge or pile']
---
bank%1:14:01:: :  [['operated', 'bank', 'objects', 'arrangement', 'similar', 'he', 'switches', 'tiers', 'row'], 'an arrangement of similar objects in a row or in tiers']
---
bank%1:21:00:: :  [['emergencies', 'reserve', 'stock', 'supply', 'especially', 'future', 'use', 'held'], 'a supply or stock held in reserve for future use (especially in emergencies)']
---
bank%1:21:01:: :

In [ ]:
syns = wn.synsets('bank', pos='n')
    
for s in syns:
    print('--->',s.name())
    print(s.examples())
    for l in s.lemmas():
        print(l.key())
        print(l)

# Skip gram model 

In [ ]:
settings = {
	'window_size': 2	# context window +- center word
	'n': 10,		# dimensions of word embeddings, also refer to size of hidden layer
	'epochs': 50,		# number of training epochs
	'learning_rate': 0.01	# learning rate
}

In [4]:
!brew install wget

!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

To reinstall 1.20.3_2, run `brew reinstall wget`
--2020-08-13 16:01:15--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.232.213
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.232.213|:443... connected.
HTTP request sent, awaiting response... 206 Partial Content
Length: 1647046227 (1.5G), 1537628778 (1.4G) remaining [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[+==================>]   1.53G  1.39MB/s    in 19m 6s  

2020-08-13 16:20:22 (1.28 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [40]:
import gensim.downloader as api
wv = api.load("glove-wiki-gigaword-50")
wv.most_similar("glass")


[('plastic', 0.7942505478858948),
 ('metal', 0.770871639251709),
 ('walls', 0.7700636386871338),
 ('marble', 0.7638523578643799),
 ('wood', 0.7624280452728271),
 ('ceramic', 0.7602593302726746),
 ('pieces', 0.7589111924171448),
 ('stained', 0.7528817057609558),
 ('tile', 0.748193621635437),
 ('furniture', 0.7463858723640442)]

In [27]:
word = 'bank'
print(model.similarity(word,'river'))
print(model.similarity(word,'financial'))
print(model.similarity(word,'institute'))

0.4142035
0.7670275
0.40241227


# getting new sentences and simialrity se divide 

In [42]:
def sketch_engine(word,pos):
    import requests
    USERNAME = 'Tushita'
    API_KEY = 'e3b4611e282c4266abe1c08d449e48cf'
    base_url = 'https://api.sketchengine.eu/bonito/run.cgi'
    string = 'q[lempos_lc="'+'('+word+')'+'-'+pos+'"]'
    data = {
     'corpname': 'preloaded/ententen15_tt21',
     'format': 'json',
      'q': [string, 'e300' ],
     'viewmode':'sen',
     #'async':'0',
     'pagesize':'500',
     #'fromp':'50',
     #'kwicleftctx ':'-1:s',
     #'kwicrightctx':'1:s'
    }
    d = requests.get(base_url + '/view?corpname=%s' % data['corpname'], params=data, auth=(USERNAME, API_KEY)).json()
    return(d)
    #print("There are %d grammar relations for %s%s (lemma+PoS) in corpus %s." % (len(d['Gramrels']), data['lemma'], data['lpos'], data['corpname']))

In [43]:
def expand_vocab(word,pos):
    d = sketch_engine(word,pos)
    expanded_vocab = []
    for line in d['Lines']:
            sentence = ''
            if(line['Left']!=[] and line["Right"]!=[]):
                left = ''
                right = ''
                for c in line['Left']:
                    if c['class']=='':
                        left = c['str'] 
                for c in line['Right']:
                    if c['class']=='':
                        right = c['str']
                centre = line['Kwic'][0]['str']
                #print(left,"<",centre,">",right)
                sentence = left+right
            elif(line['Left']==[]):
                right = ''
                for c in line['Right']:
                    if c['class']=='':
                        right = c['str']
                centre = line['Kwic'][0]['str']
                #print("<",centre,">",right)
                sentence = right
            elif(line["Right"]==[]):
                left = ''
                for c in line['Left']:
                    if c['class']=='':
                        left = c['str'] 
                centre = line['Kwic'][0]['str']
                #print(left,"<",centre,">")
                sentence = left
            normal = normalize(sentence)
            #print(normal)
            expanded_vocab.append([sentence,list(set(normal))])
            return(expanded_vocab)
            #print('----')

In [44]:
def expand_context(word, pos):
    neighbour = wordnet(word,pos)
    if pos=='a':
        expanded_vocab = expand_vocab(word,'j')
    else:
        expanded_vocab = expand_vocab(word,pos)
    expanded_context = neighbour 
    for random in expanded_vocab:
        #print(random[0])
        #print(random[1])
        temp_cal = {}
        for n, l in neighbour.items():
            name = n
            temp = l[0]
            #print("-->",temp)
            total = 0 
            for r in random[1]:
                score_temp = []
                score = 0
                for t in temp:
                    if(r in wv.vocab and t in wv.vocab):
                        score_temp.append(wv.similarity(r, t))
                        #print(r,t,wv.similarity(r, t))
                if score_temp!=[]:
                    score = max(score_temp)
                #print(score)
                total+=score
            temp_cal[name] = total
        #print(temp_cal)
        max_score = max(temp_cal,key=temp_cal.get)
        #print(neighbour[max_score][1])
        temp_list = expanded_context[max_score][0]
        temp_list.extend(random[1])
        expanded_context[max_score][0] = list(set(temp_list))
        return(expanded_context)
        #print("-----")

# predicting

In [ ]:
"""tree["id"] = child.get('id')
    tree['lemma'] = child.get('lemma')
    tree['partOfSpeech'] = child.get('partOfSpeech')
    tree['token'] = child.get('token')
    tree['tokenEnd'] = child.get('tokenEnd')
    tree['tokenStart'] = child.get('tokenStart')
    tree['text'] = child.text"""

In [125]:
f = open('WSD_Unified_Evaluation_Datasets/semeval-2012-task-13-trial-data/evaluation/unsupervised/output.key','a')
for name, item in d_tree.items():
    target_sentence = item['text']
    target_start = int(item['tokenStart'])
    target_end = int(item['tokenEnd'])
    target_word = target_sentence[target_start:target_end].lower()
    print(target_sentence[target_start:target_end])
    print(target_sentence)
    print("target--->",item['lemma'])
    sents = normalize(target_sentence)
    #print(sents)
    """#give distance value 
    target_idx = sents.index(target_word)
    left_dist = sents[:target_idx]
    if target_idx+1!=len(target_sentence):
        right_dist = sents[target_idx+1:len(target_sentence)]
    context_sents = []
    if len(left_dist)!=0 and len(right_dist)!=0:
        length = min(len(left_dist),len(right_dist))
        for i in range(length):
            context_sents.append([left_dist[i],i+1])
        for i in range(length):
            context_sents.append([right_dist[i],length-i])
        print(context_sents)
    else:
        if len(left_dist)==0:
            for i in range(len(right_dist)):
                context_sents.append([right_dist[i],len(right_dist)-i])
            print(context_sents)
        elif len(right_dist)==0:
            for i in range(len(left_dist)):
                context_sents.append([left_dist[i],i+1])
            print(context_sents)"""
    
    
    #print(target_idx)
    if item['lemma']!='different':
        context_dict = expand_context(item['lemma'],item['partOfSpeech'])
    else:
        context_dict = expand_context('different','a')

    score_dict = {}
    for name, l in context_dict.items():
        temp = l[0]
        total = 0
        for s in sents:
            score_temp = []
            score = 0
            for t in temp:
                if(s in wv.vocab and t in wv.vocab):
                    score_temp.append(wv.similarity(s, t))
            if score_temp!=[]:
                score = max(score_temp)
            total += (score)
        score_dict[name] = total
    total_sum = sum(score_dict.values())
    s = ''
    for x, y in score_dict.items():
        y = y/total_sum
        score_dict[x] = y
        s += x+'/'+str(y)+' '
    key = max(score_dict,key=score_dict.get)
    print("definition allotted: ",context_dict[key][1])
    print(item['lemma']+'.'+item['partOfSpeech']+' '+item['id']+' ',s)
    file_string = item['lemma']+'.'+item['partOfSpeech']+' '+item['id']+' '+s+'\n'
    f.write(file_string)
    print("-------")
f.close()

add
Fry in butter a small minced onion, rub with a tablespoonful of flour, add half a cup of cream, six beaten eggs, pepper, celery salt, a teaspoonful of minced chives, a dash of cayenne, and a pinch of nutmeg.
target---> add
definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity, size or scope of
add.v br-e11.snum=49  add%2:30:00::/0.2006158019311439 add%2:32:01::/0.11450448144157994 add%2:40:00::/0.134644778940827 add%2:31:00::/0.1856758130638118 add%2:32:00::/0.18115062797695544 add%2:42:00::/0.18340849664568193 
-------
adding
Check thickness of clay and build up thin areas by moistening surface with a little water and adding small pieces of clay.
target---> add
definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity, size or scope of
add.v br-e15.snum=18  add%2:30:00::/0.18373945355690838 add%2:32:01::/0.15488593788562435 add%2:40:00::/0.17648206461284566 add%2:31:

definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity, size or scope of
add.v br-j38.snum=8  add%2:30:00::/0.19143422054536133 add%2:32:01::/0.15708538750752474 add%2:40:00::/0.17958652386262294 add%2:31:00::/0.16984044540884602 add%2:32:00::/0.141806137221564 add%2:42:00::/0.16024728545408098 
-------
added
Even more poignantly, and with the insight of a genius, Channing added - remember, this is Channing, not Parker !
target---> add
definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity, size or scope of
add.v br-d05.snum=42  add%2:30:00::/0.20197079193825962 add%2:32:01::/0.15921342505019348 add%2:40:00::/0.2017266979932903 add%2:31:00::/0.1585576683224177 add%2:32:00::/0.11269947552443 add%2:42:00::/0.16583194117140893 
-------
added
But in general the individual variation is a small thing added onto basic linguistic features of greater magnitude.
target---> add
de

definition allotted:  bestow a quality on
add.v add.v.bnc.00059115  add%2:30:00::/0.19032710568309757 add%2:32:01::/0.1503740664617608 add%2:40:00::/0.19712031984967543 add%2:31:00::/0.15186034172691587 add%2:32:00::/0.14620777007709762 add%2:42:00::/0.16411039620145274 
-------
add
To obtain a sinusoidal output,  add  the optional shaping circuit: Choose CF for optimum response at the desired frequency.
target---> add
definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity, size or scope of
add.v add.v.bnc.00067172  add%2:30:00::/0.18833074928233717 add%2:32:01::/0.13519330622741055 add%2:40:00::/0.17155535358030918 add%2:31:00::/0.17845087764541873 add%2:32:00::/0.16591768307759783 add%2:42:00::/0.16055203018692654 
-------
added
Veronica  added  that she'd put away later and went to join the others.
target---> add
definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity

definition allotted:  make an addition (to); join or combine or unite with others; increase the quality, quantity, size or scope of
add.v add.v.bnc.00014126  add%2:30:00::/0.1853751920064139 add%2:32:01::/0.16363709225440343 add%2:40:00::/0.18130210985236417 add%2:31:00::/0.16844620531214222 add%2:32:00::/0.15001442371526866 add%2:42:00::/0.15122497685940764 
-------
argument
Changes in the basic wage rate are cost raising, and they constitute an argument for raising prices.
target---> argument
definition allotted:  a fact or assertion offered as evidence that something is true
argument.n br-j41.snum=63  argument%1:10:02::/0.14639955963502463 argument%1:10:03::/0.14584410078238552 argument%1:10:00::/0.1456746774187322 argument%1:10:01::/0.1443359235020602 argument%1:10:04::/0.14025831970424993 argument%1:09:00::/0.13792839836764365 argument%1:09:01::/0.13955902058990385 
-------
argument
Obviously, a satisfactory answer to the third question is imperative, if the argument is to get und

definition allotted:  a fact or assertion offered as evidence that something is true
argument.n br-f08.snum=45  argument%1:10:02::/0.1663106672952353 argument%1:10:03::/0.1473262005814953 argument%1:10:00::/0.154946555175638 argument%1:10:01::/0.14182865598166236 argument%1:10:04::/0.11825817616096508 argument%1:09:00::/0.12752383663610672 argument%1:09:01::/0.14380590816889727 
-------
argument
The functions f and b have exactly the same multiplicity at every argument t.
target---> argument
definition allotted:  a variable in a logical or mathematical expression whose value determines the dependent variable; if f(x)=y, x is the independent variable
argument.n br-j20.snum=90  argument%1:10:02::/0.14274329625664373 argument%1:10:03::/0.13684594278944537 argument%1:10:00::/0.14066581244852802 argument%1:10:01::/0.14501430413012814 argument%1:10:04::/0.13560927303282455 argument%1:09:00::/0.15825741720851336 argument%1:09:01::/0.14086395413391684 
-------
argument
The argument against thi

...

definition allotted:  a medium for written communication
paper.n br-j03.snum=72  paper%1:27:00::/0.14281271042514113 paper%1:10:01::/0.14169900316770093 paper%1:10:03::/0.13608343528841568 paper%1:10:00::/0.1621224357295817 paper%1:10:02::/0.15064364521866946 paper%1:14:00::/0.11098083122262982 paper%1:06:00::/0.15565793894786126 
-------
paper
The figures on the worksheet paper in front of her were jumping and waving around so badly it was all she could do to make them out clearly enough to copy them with the typewriter.
target---> paper
definition allotted:  a medium for written communication
paper.n br-l10.snum=112  paper%1:27:00::/0.13903508052156652 paper%1:10:01::/0.14728382525572745 paper%1:10:03::/0.14175059160631434 paper%1:10:00::/0.15136841465909923 paper%1:10:02::/0.14526878465677087 paper%1:14:00::/0.12859642233327234 paper%1:06:00::/0.14669688096724925 
-------
paper
A second explanation is suggested by the material described in Rowlands' paper .
target---> paper
definiti

definition allotted:  the physical object that is the product of a newspaper publisher
paper.n paper.n.bnc.00101869  paper%1:27:00::/0.15467994432656304 paper%1:10:01::/0.13296009438855755 paper%1:10:03::/0.14412260464834387 paper%1:10:00::/0.14321495434503628 paper%1:10:02::/0.14256575682780545 paper%1:14:00::/0.12729893985231494 paper%1:06:00::/0.1551577056113789 
-------
paper
They do not make the distinction between an image on canvas or panel and one on  paper  which seems so obvious indeed axiomatic to Europeans.
target---> paper
definition allotted:  a medium for written communication
paper.n paper.n.bnc.00092455  paper%1:27:00::/0.14632963587625478 paper%1:10:01::/0.14325987803259987 paper%1:10:03::/0.12999296240536973 paper%1:10:00::/0.15448705455083864 paper%1:10:02::/0.1489843891870979 paper%1:14:00::/0.12535769481315062 paper%1:06:00::/0.1515883851346885 
-------
paper
Papersave sells recycled  paper  for everything from quality stationery, to 
target---> paper
definition a

definition allotted:  a material made of cellulose pulp derived mainly from wood or rags or certain grasses
paper.n paper.n.bnc.00109246  paper%1:27:00::/0.1573698305462222 paper%1:10:01::/0.13708461648913323 paper%1:10:03::/0.1408816065941636 paper%1:10:00::/0.14559629208721606 paper%1:10:02::/0.13946321341748835 paper%1:14:00::/0.13274909176672894 paper%1:06:00::/0.1468553490990476 
-------
papers
Some trial and error should eliminate the laziest  papers , whose critics do no more than pillage catalogue introductions, or even press releases.
target---> paper
definition allotted:  a scholarly article describing the results of observations or stating hypotheses
paper.n paper.n.bnc.00003530  paper%1:27:00::/0.14673012896850823 paper%1:10:01::/0.13762014112822238 paper%1:10:03::/0.14377449418830027 paper%1:10:00::/0.14719882366603568 paper%1:10:02::/0.15073349927009594 paper%1:14:00::/0.130310742392635 paper%1:06:00::/0.14363217038620252 
-------
paper
The  paper  quotes documents which 

definition allotted:  win something through one's efforts
win.v br-b22.snum=73  win%2:33:00::/0.2487416473351572 win%2:40:00::/0.26087065399449166 win%2:33:01::/0.23244557483452724 win%2:41:04::/0.2579421238358239 
-------
win
To win her favors, her husband first took an additional job, then desperately began to embezzle from his employer.
target---> win
definition allotted:  attain success or reach a desired goal
win.v br-f08.snum=36  win%2:33:00::/0.24848580209726154 win%2:40:00::/0.24244859260412033 win%2:33:01::/0.2346130153375442 win%2:41:04::/0.27445258996107397 
-------
winning
At a ceremony in the reception room of Mayor Richardson Dilworth, the Eagles were honored for winning the championship.
target---> win
definition allotted:  obtain advantages, such as points, etc.
win.v br-a14.snum=22  win%2:33:00::/0.2518024786447669 win%2:40:00::/0.23581372999929698 win%2:33:01::/0.25939333542563303 win%2:41:04::/0.2529904559303031 
-------
win
Although both concede they would like to h

definition allotted:  be the winner in a contest or competition; be victorious
win.v win.v.bnc.00074845  win%2:33:00::/0.2603404233599222 win%2:40:00::/0.2564205136189793 win%2:33:01::/0.23017156157828203 win%2:41:04::/0.2530675014428165 
-------
win
Perhaps he thought I was the impersonator, dressed as a man, hoping to  win  his confidence and thus learn the kind of mannerisms and opinions he might reveal only in private.
target---> win
definition allotted:  win something through one's efforts
win.v win.v.bnc.00047697  win%2:33:00::/0.25744993153813184 win%2:40:00::/0.26324546891975137 win%2:33:01::/0.217273633104918 win%2:41:04::/0.26203096643719875 
-------
won
My bandsaw competes with the radial-arm for clatters and bangs so the big router  won  with a large chamfer bit.
target---> win
definition allotted:  attain success or reach a desired goal
win.v win.v.bnc.00050973  win%2:33:00::/0.2247219727174337 win%2:40:00::/0.2459370660312523 win%2:33:01::/0.25073382700291597 win%2:41:04:

In [129]:
sentence1= "There is a financial institute at the river bank"
sentence2= "I am going to the bank to draw some money"
sentence = sentence2
target = 'bank'
pos = pos_tag(sentence,target)

target_sentence = sentence
target_word = target
print("target--->",target_word)
sents = normalize(target_sentence)
#print(sents)
#give distance value 
target_idx = sents.index(target_word)
left_dist = sents[:target_idx]
if target_idx+1!=len(target_sentence):
    right_dist = sents[target_idx+1:len(target_sentence)]
context_sents = []
if len(left_dist)!=0 and len(right_dist)!=0:
    length = min(len(left_dist),len(right_dist))
    for i in range(length):
        context_sents.append([left_dist[i],i+1])
    for i in range(length):
        context_sents.append([right_dist[i],length-i])
    #print(context_sents)
else:
    if len(left_dist)==0:
        for i in range(len(right_dist)):
            context_sents.append([right_dist[i],len(right_dist)-i])
        #print(context_sents)
    elif len(right_dist)==0:
        for i in range(len(left_dist)):
            context_sents.append([left_dist[i],i+1])
        #print(context_sents)

already_done = {}
#print(target_idx)
if target_word not in already_done:
    if target_word!='different':
        cont_dict = expand_context(target_word,pos)
    else:
        cont_dict = expand_context('different','a')
    already_done[target_word] = cont_dict

context_dict = already_done[target_word]

score_dict = {}
for name, l in context_dict.items():
    temp = l[0]
    total = 0
    for s in context_sents:
        score_temp = []
        score = 0
        for t in temp:
            if(s[0] in wv.vocab and t in wv.vocab):
                score_temp.append(wv.similarity(s[0], t))
        if score_temp!=[]:
            score = max(score_temp)
        total += (score*s[1])
    score_dict[name] = total
total_sum = sum(score_dict.values())
s = ''
for x, y in score_dict.items():
    y = y/total_sum
    score_dict[x] = y
    s += x+'/'+str(y)+' '
key = max(score_dict,key=score_dict.get)
print("definition: ",context_dict[key][1])
print(target_word+'.'+pos+' ',s)

print("-------")

I -PRON- PRON PRP nsubj X True True
am be AUX VBP aux xx True True
going go VERB VBG ROOT xxxx True False
to to ADP IN prep xx True True
the the DET DT det xxx True True
bank bank NOUN NN pobj xxxx True False
to to PART TO aux xx True True
draw draw VERB VB advcl xxxx True False
some some DET DT det xxxx True True
money money NOUN NN dobj xxxx True False
target---> bank
definition:  a financial institution that accepts deposits and channels the money into lending activities
bank.n  bank%1:17:01::/0.10710796892065247 bank%1:14:00::/0.10952270512786152 bank%1:17:00::/0.09262113330010091 bank%1:14:01::/0.09405537600755969 bank%1:21:00::/0.09225329066610059 bank%1:21:01::/0.10854543046658187 bank%1:17:02::/0.10733772255036678 bank%1:06:01::/0.10468799584670882 bank%1:06:00::/0.08531607667886984 bank%1:04:00::/0.09855230043519753 
-------
